In [1]:
import itertools
import json
import os
import pathlib
import pprint
import re
import shutil
import warnings

from np_session import Session

def eat_warning_lines(msg:str, category:str, *args, **kwargs):
    print("/n{}: {}/n".format(category.__name__, msg))
warnings.showwarning = eat_warning_lines

INCOMING_ROOT = pathlib.Path("//allen/programs/braintv/production/incoming/neuralcoding")
def opt_root_from_mouse_id(mouse_id:str|int) -> pathlib.Path|None:
    openscope_root = pathlib.Path("//allen/programs/mindscope/workgroups/openscope")
    if (root := tuple(openscope_root.glob(f"*/AlignToPhysiology/{mouse_id}"))):
        return root[0]
    return None

In [2]:
mice = (
668800,
669417,
673694,
)
mouse_dirs = [opt_root_from_mouse_id(m) for m in mice]
print(mouse_dirs)

[WindowsPath('//allen/programs/mindscope/workgroups/openscope/OPT_BarC/AlignToPhysiology/668800'), WindowsPath('//allen/programs/mindscope/workgroups/openscope/OPT_BarC/AlignToPhysiology/669417'), WindowsPath('//allen/programs/mindscope/workgroups/openscope/OPT_BarC/AlignToPhysiology/673694')]


In [3]:
def most_recent_file(paths: list[pathlib.Path]) -> pathlib.Path | None:
    return max(paths, key=os.path.getmtime) if paths else None
def matching_files_by_probe(probe: str, paths: list[pathlib.Path]) -> list[pathlib.Path]:
    return [p for p in paths if f'_probe{probe.upper()}' in p.name]
def most_recent_file_each_probe(paths: list[pathlib.Path]) -> list[pathlib.Path]:
    files_or_nones = [most_recent_file(matching_files_by_probe(p, paths)) for p in 'ABCDEF']
    return [_ for _ in files_or_nones if _ is not None]

In [4]:
for mouse_dir in mouse_dirs:
    mouse_id = mouse_dir.parts[-1]

    src_paths = (
        list(mouse_dir.glob('images/*_probe*sortedccf_regions.csv'))
        + list(mouse_dir.glob('images/*_probe*sorted_ccf_regions.csv'))
    )
    src_paths = most_recent_file_each_probe(src_paths)
    
    if not src_paths:
        print(f"No csv files found for {mouse_id} in {mouse_dir}")
        continue
    
    if len(src_paths) > 6:
        print(f"Too many csv files found for {mouse_id} in {mouse_dir}: {[p.name for p in src_paths]}")
        continue 
    
    session_id = src_paths[0].name.split('_')[0]
    try:
        lims_path = Session(f"{session_id}_366122_12345678").lims_path 
        # fill in dummy mouse_id/date - only need session id to be correct
        if not lims_path:
            raise Exception
    except:
        print(f"Could not find LIMS path for session {session_id}")
        continue
    
    # lims_probe_folder = f'{session_id}_probe{probe_letter}'
    if lims_paths := list(lims_path.glob(f'*/*_probe*/continuous/Neuropix-PXI-100.0/ccf_regions.csv')):
        print(f"{mouse_id}: found {len(lims_paths)} csv files on lims for {session_id}")       
        pprint.pprint(lims_paths)
        response = None
        while not (response == 'y' or response == 'n'):
            response = input("Existing ccf_regions.csv on lims - continue and overwrite? (y/n): ")
            if response == 'y':
                break
        else:
            print(f"Skipping {session_id} {mouse_id}\n")
            continue
    
    platform_files = {'files': {}}
    for src_path in src_paths:
    
        # get probe letter 
        probe_letter = re.findall('_probe([A-F])', src_path.name)[0]
        
        incoming_probe_folder = f'{session_id}_probe{probe_letter}_sorted'
        probe_key = f"ephys_raw_data_probe_{probe_letter}_sorted"
            
        # copy each csv to a common folder in incoming 
        dest_path = INCOMING_ROOT / incoming_probe_folder / 'continuous'/ 'Neuropix-PXI-100.0' / 'ccf_regions.csv'
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src_path, dest_path)

        if not (INCOMING_ROOT / incoming_probe_folder).exists():
            raise FileNotFoundError(f"Could not find {incoming_probe_folder} in {INCOMING_ROOT}")
            
        # add probe entry to platform json dict
        platform_files['files'].update(
            {probe_key:
                {"directory_name":incoming_probe_folder}
                }
            )
    
    print(f"{mouse_id}: found {len(src_paths)} csv files in {mouse_dir}")       
    pprint.pprint(platform_files) if platform_files['files'] else None
    response = None
    while not (response == 'y' or response == 'n'):
        response = input("Check the probe entries look correct - continue? (y/n): ")
        if response == 'y':
            break
    else:
        print(f"Skipping {session_id} {mouse_id}\n")
        continue
    
    # rename any existing platform jsons in incoming, so we don't trigger their
    # uploads prematurely
    for existing_platform_json in INCOMING_ROOT.glob(f'{session_id}*_platform*.json'):
        existing_platform_json.replace(existing_platform_json.with_suffix('.bak'))
        print(f"renamed {existing_platform_json}")
    
    # write the D2 platform json to incoming 
    with open(INCOMING_ROOT / f"{session_id}_platformD2.json", 'w') as f:
        json.dump(platform_files, f, indent=4)
    print(f"\twritten {INCOMING_ROOT / f'{session_id}_platformD2.json'}")
    
    # write a trigger file to incoming/trigger
    with open(INCOMING_ROOT / 'trigger' / f"{session_id}.ecp", 'w') as f:
        f.writelines('sessionid: ' + session_id + "\n")
        f.writelines("location: '" + INCOMING_ROOT.as_posix() + "'")
    print(f"\twritten {INCOMING_ROOT / 'trigger' / f'{session_id}.ecp'}")

668800: found 6 csv files in \\allen\programs\mindscope\workgroups\openscope\OPT_BarC\AlignToPhysiology\668800
{'files': {'ephys_raw_data_probe_A_sorted': {'directory_name': '1274061513_probeA_sorted'},
           'ephys_raw_data_probe_B_sorted': {'directory_name': '1274061513_probeB_sorted'},
           'ephys_raw_data_probe_C_sorted': {'directory_name': '1274061513_probeC_sorted'},
           'ephys_raw_data_probe_D_sorted': {'directory_name': '1274061513_probeD_sorted'},
           'ephys_raw_data_probe_E_sorted': {'directory_name': '1274061513_probeE_sorted'},
           'ephys_raw_data_probe_F_sorted': {'directory_name': '1274061513_probeF_sorted'}}}
renamed \\allen\programs\braintv\production\incoming\neuralcoding\1274061513_668800_20230601_platformD1.json
	written \\allen\programs\braintv\production\incoming\neuralcoding\1274061513_platformD2.json
	written \\allen\programs\braintv\production\incoming\neuralcoding\trigger\1274061513.ecp
669417: found 6 csv files in \\allen\progra

***
A `.ecp` trigger file for each mouse should now be in this folder (for about a minute,
before lims sees the trigger file, starts the job, and removes the `.ecp`)

In [5]:
os.startfile(INCOMING_ROOT / 'trigger')